# Homework 1


In [43]:
# Import necessary libraries
from openai import OpenAI
import os
import requests
from elasticsearch import Elasticsearch
import tiktoken

***Q1. Running Elastic***

What's the version.build_hash value?

In [ ]:
# Get the cluster information
!curl localhost:9200

{
  "name" : "5e72f9963382",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "o6f-JGvpQoego2PpIJrQ8A",
  "version" : {
    "number" : "8.17.6",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "dbcbbbd0bc4924cfeb28929dc05d82d662c527b7",
    "build_date" : "2025-04-30T14:07:12.231372970Z",
    "build_snapshot" : false,
    "lucene_version" : "9.12.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [ ]:
# Getting the data
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

***Q2. Indexing the data***

Which function do you use for adding your data to elastic?

In [6]:
# Index the data using Elasticsearch
es_client = Elasticsearch('http://localhost:9200')
# Define the index mapping
mapping = {
    "mappings": {
        "properties": {
            "course": {"type": "keyword"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "text": {"type": "text"}
        }
    }
}

# Create the index with the mapping (ignore if already exists)
if not es_client.indices.exists(index="documents"):
    es_client.indices.create(index="documents", body=mapping)

# Index each document
for doc in documents:
    es_client.index(index="documents", document=doc)

/var/folders/_9/rjnbrnf56dld9s5r75l__p6r0000gn/T/ipykernel_28088/210292929.py:17: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es_client.indices.create(index="documents", body=mapping)


***Q3. Searching***

What's the score for the top ranking result?

In [ ]:
# Function to search the indexed data
def search_documents(query):
    query_body = {
        "query": {
            "multi_match": {
                "query": query,
                "type": "best_fields",
                "fields": ["text", "question^4"]
            }
        }
    }

    response = es_client.search(
        index="documents",
        body=query_body
    )
    return response['hits']['hits']


In [ ]:
# Get only _score for top results
query = "How do execute a command on a Kubernetes pod?"
results = search_documents(query)
print(results[0]['_score'])


44.50556


/var/folders/_9/rjnbrnf56dld9s5r75l__p6r0000gn/T/ipykernel_28088/2380535908.py:13: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  response = es_client.search(


***Q4. Filtering***

What's the 3rd question returned by the search engine?

In [33]:
def search_documents(query, size=5):
    query_body = {
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "type": "best_fields",
                        "fields": ["text", "question^4"]
                    }
                },
                "filter": [
                    {"term": {"course": "machine-learning-zoomcamp"}}
                ]
            }
        }
    }

    response = es_client.search(
        index="documents",
        body=query_body,
        size=size
    )
    return response['hits']['hits']

In [35]:
# Print the question of the third result for a specific query
query = "How do copy a file to a Docker container?"
results = search_documents(query, size=3)
print(results[2]['_source']['question'])

How do I copy files from a different folder into docker container’s working directory?


/var/folders/_9/rjnbrnf56dld9s5r75l__p6r0000gn/T/ipykernel_28088/171566568.py:19: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  response = es_client.search(


***Q5. Building a prompt***

What's the length of the resulting prompt? 

In [36]:
# Define the context template for the LLM

context_template = """
Q: {question}
A: {text}
""".strip()

# Prepare the context for the LLM
context = ""
for result in results:
    context += context_template.format(
        question=result['_source']['question'],
        text=result['_source']['text']
    ) + "\n\n"

In [37]:
# Define the prompt template for the LLM
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

# Prepare the prompt for the LLM
prompt = prompt_template.format(
            question=query,
            context=context
        ).strip()

In [38]:
# Print the length of the prompt
print("Length of the prompt:", len(prompt))

Length of the prompt: 1446


***Q6. Tokens***

How many tokens does our prompt have?

In [42]:
#!pip install tiktoken

In [50]:
# Encode the prompt using tiktoken
# Initialize the tokenizer for the GPT-4o model
encoding = tiktoken.encoding_for_model("gpt-4o")
# Encode the prompt and 
tokens = encoding.encode(prompt)
# Print the number of tokens
print(len(tokens))

320


In [51]:
# Display first 20 tokens
print("First 20 tokens:", tokens[:20])

First 20 tokens: [63842, 261, 4165, 14029, 29186, 13, 30985, 290, 150339, 4122, 402, 290, 31810, 8099, 591, 290, 40251, 7862, 558, 8470]


In [49]:
# Decode a single token into a word - tocken 63842
encoding.decode_single_token_bytes(63842)

b"You're"